In [5]:
import pyodbc 

In [8]:
cnxn = pyodbc.connect("Driver={SQL Server};"
                      "Server=LAPTOP-5IT4U5OF\SQLEXPRESS;"
                      "Database=Hotelmenu;"
                      "Trusted_Connection=yes;")

In [9]:
cursor=cnxn.cursor()

In [10]:
cursor.execute('SELECT  * from menu')

In [11]:
records=cursor.fetchall()
for i in records:
    print(i)

(1, 'Veg Biryani', Decimal('180.00'))
(2, 'Chicken Biryani', Decimal('250.00'))
(3, 'Paneer Butter Masala', Decimal('220.00'))
(4, 'Butter Naan', Decimal('40.00'))
(5, 'Masala Dosa', Decimal('120.00'))
(6, 'Idli Sambar', Decimal('80.00'))
(7, 'Veg Fried Rice', Decimal('150.00'))
(8, 'Chicken Curry', Decimal('240.00'))
(9, 'Cold Drink', Decimal('50.00'))
(10, 'Ice Cream', Decimal('90.00'))


In [12]:
cursor.execute("SELECT * FROM menu")

columns = [column[0] for column in cursor.description]
print("Columns:", columns)

records = cursor.fetchall()

for row in records:
    print(row)

Columns: ['ItemID', 'ItemName', 'Price']
(1, 'Veg Biryani', Decimal('180.00'))
(2, 'Chicken Biryani', Decimal('250.00'))
(3, 'Paneer Butter Masala', Decimal('220.00'))
(4, 'Butter Naan', Decimal('40.00'))
(5, 'Masala Dosa', Decimal('120.00'))
(6, 'Idli Sambar', Decimal('80.00'))
(7, 'Veg Fried Rice', Decimal('150.00'))
(8, 'Chicken Curry', Decimal('240.00'))
(9, 'Cold Drink', Decimal('50.00'))
(10, 'Ice Cream', Decimal('90.00'))


In [13]:
print(cursor.description)

(('ItemID', <class 'int'>, None, 10, 10, 0, False), ('ItemName', <class 'str'>, None, 100, 100, 0, False), ('Price', <class 'decimal.Decimal'>, None, 10, 10, 2, False))


In [12]:
import pyodbc
import datetime
class Food_Orders:
    menu_key = []

    def __init__(self):
        self.order_list = []
        self.menu = {}
        self.submenu = {}

        try:
            
            self.cnxn = pyodbc.connect(
                "Driver={SQL Server};"
                "Server=LAPTOP-5IT4U5OF\\SQLEXPRESS;"
                "Database=Hotelmenu;"
                "Trusted_Connection=yes;"
            )
            self.cursor = self.cnxn.cursor()
            self.cursor.execute('SELECT * FROM menu')
            records = self.cursor.fetchall()

            
            for i in records:
                self.submenu = {i[0]: {"name": i[1], "price": i[2]}}
                self.menu.update(self.submenu)

            print("Here is the Menu list:", self.menu)
            print("Please select the menu_key")
            self.take_input()

        
        except Exception as e:
            print("An error occurred:", e)
        # finally:
            
        #         cursor.close()
        #         cnxn.close()
            

    def take_input(self):
        print("Here is the Menu list:", self.menu)

        while True:
            try:
                key = int(input("Enter key (0 to finalize): "))
            except ValueError:
                print("Please enter a valid number")
                continue

            if key == 0:
                break

            if key in self.menu:
                self.menu_key.append(key)
                print("Menu list is", self.menu_key)
            else:
                print("Invalid key")

        self.take_order(self.menu_key)

    def take_order(self, menu_key):
        self.order_list = [self.menu.get(i).get('name') for i in menu_key]
        print("Selected items are:")
        for item in self.order_list:
            print(item)
        self.generate_bill(menu_key)
        self.deliver_order()

    def deliver_order(self):
        print("Your ORDER is delivered")

    def generate_bill(self, menu_key):
        self.order_bill = [self.menu.get(i).get('price') for i in menu_key]
        total_bill = sum(self.order_bill)
        print("Total bill amount is:", total_bill)
        self.bill_layout(self.order_list, self.order_bill)

    def bill_layout(self, order_list, order_bill):
        order_time = datetime.datetime.now() 
        order_id = order_time.strftime("%Y%m%d%H%M%S") 
        total = sum(order_bill)
        # print("|" + "-"*60 + "|")
        # print("|{:^60}|".format("Welcome to Hotel Shinde's"))
        # print("|" + "-"*60 + "|")
        # print("|{:<30}{:>30}|".format("Order id :", "Receipt"))
        # print("|" + "-"*60 + "|")
        # print("|{:<10}{:<21}{:>10}{:>19}|".format("sr", "Food", "Quant", "Price"))
        # #  a=len(order_bill)  
        # #  for i in range(1,a+1): print("|{:<6}{:<25}{:>10}{:>19}|".format(i,order_list[i-1],"1",order_bill[i-1])) 
        # #  total=total+order_bill[i-1]
        # for i, (food, price) in enumerate(zip(order_list, order_bill), 1):
        #     print("|{:<6}{:<25}{:>10}{:>19}|".format(i, food, "1", price))
        # print("|" + "-"*60 + "|")
        # print("|{:>15}{:>45}|".format("Total", total))
        # print("|" + "-"*60 + "|")
        
        lines = []
        lines.append("|" + "-"*60 + "|")
        lines.append("|{:^60}|".format("Welcome to Hotel Shinde's"))
        lines.append("|" + "-"*60 + "|")
        lines.append("|{:<10}{:<30}{:>20}|".format("Order id :",order_id, "Receipt"))
        lines.append("|" + "-"*60 + "|")
        lines.append("|{:<10}{:<21}{:>10}{:>19}|".format("sr", "Food", "Quant", "Price"))

        for i, (food, price) in enumerate(zip(order_list, order_bill), 1):
            lines.append("|{:<6}{:<25}{:>10}{:>19}|".format(i, food, "1", price))

        lines.append("|" + "-"*60 + "|")
        lines.append("|{:>15}{:>45}|".format("Total", total))
        lines.append("|" + "-"*60 + "|")
        self.savebilltofile(lines,order_id)
        self.save_to_database(order_id)
        
    def savebilltofile(self,lines,order_id):
        with open(f"C:/Users/Harsh/Desktop/{order_id}.txt", "w") as f:
            for line in lines:
                    f.write(line + "\n")
            print(f"\nBill has been saved to '{order_id}.txt' successfully!")

        self.printbill(order_id)

    def printbill(self,order_id):
        try:
             with open(f"C:/Users/Harsh/Desktop/{order_id}.txt", "r") as f:
                print("\nPrinting Bill:\n")
                for line in f:
                    print(line, end="")  

        except FileNotFoundError:
             print(f"Bill file '{order_id}.txt' not found!")

    def save_to_database(self, order_id):
      try:
        for item, price in zip(self.order_list, self.order_bill):
            self.cursor.execute(
                "INSERT INTO ORDER_DETAILS (ORDERID, MENUITEM, PRICE) VALUES (?, ?, ?)",
                order_id, item, price
            )

        self.cnxn.commit()
        print("\nOrder inserted successfully into database!")

      except Exception as e:
        print("Database Error:", e)

      finally :
        self.cursor.close()
        self.cnxn.close()





In [13]:
obj=Food_Orders()

Here is the Menu list: {1: {'name': 'Veg Biryani', 'price': Decimal('180.00')}, 2: {'name': 'Chicken Biryani', 'price': Decimal('250.00')}, 3: {'name': 'Paneer Butter Masala', 'price': Decimal('220.00')}, 4: {'name': 'Butter Naan', 'price': Decimal('40.00')}, 5: {'name': 'Masala Dosa', 'price': Decimal('120.00')}, 6: {'name': 'Idli Sambar', 'price': Decimal('80.00')}, 7: {'name': 'Veg Fried Rice', 'price': Decimal('150.00')}, 8: {'name': 'Chicken Curry', 'price': Decimal('240.00')}, 9: {'name': 'Cold Drink', 'price': Decimal('50.00')}, 10: {'name': 'Ice Cream', 'price': Decimal('90.00')}}
Please select the menu_key
Here is the Menu list: {1: {'name': 'Veg Biryani', 'price': Decimal('180.00')}, 2: {'name': 'Chicken Biryani', 'price': Decimal('250.00')}, 3: {'name': 'Paneer Butter Masala', 'price': Decimal('220.00')}, 4: {'name': 'Butter Naan', 'price': Decimal('40.00')}, 5: {'name': 'Masala Dosa', 'price': Decimal('120.00')}, 6: {'name': 'Idli Sambar', 'price': Decimal('80.00')}, 7: {'n

Menu list is [4]
Menu list is [4, 7]
Menu list is [4, 7, 9]
Selected items are:
Butter Naan
Veg Fried Rice
Cold Drink
Total bill amount is: 240.00

Bill has been saved to '20260219141139.txt' successfully!

Printing Bill:

|------------------------------------------------------------|
|                 Welcome to Hotel Shinde's                  |
|------------------------------------------------------------|
|Order id :20260219141139                             Receipt|
|------------------------------------------------------------|
|sr        Food                      Quant              Price|
|1     Butter Naan                       1              40.00|
|2     Veg Fried Rice                    1             150.00|
|3     Cold Drink                        1              50.00|
|------------------------------------------------------------|
|          Total                                       240.00|
|------------------------------------------------------------|

Order inserted succe